In [1]:
!pip install python-mnist

In [3]:
#export
from exp.nb_01 import *
from fastai.vision.all import *
from mnist import MNIST
import pickle
import numpy as np

In [8]:
#export

def fetch(url):
    import requests, os, hashlib, tempfile
    fp = os.path.join(tempfile.gettempdir(), hashlib.md5(url.encode('utf-8')).hexdigest())
    if os.path.isfile(fp) and os.stat(fp).st_size > 0:
        with open(fp, "rb") as f:
            dat = f.read()
    else:
        print("fetching %s" % url)
        dat = requests.get(url).content
        with open(fp+".tmp", "wb") as f:
            f.write(dat)
        os.rename(fp+".tmp", fp)
    return dat

In [26]:
#export
def get_data():
    import gzip
    parse = lambda dat: np.frombuffer(gzip.decompress(dat), dtype=np.uint8).copy()
    X_train = parse(fetch("http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz"))[0x10:].reshape((-1, 28*28)).astype(np.float32)
    Y_train = parse(fetch("http://yann.lecun.com/exdb/mnist/train-labels-idx1-ubyte.gz"))[8:]
    X_test = parse(fetch("http://yann.lecun.com/exdb/mnist/t10k-images-idx3-ubyte.gz"))[0x10:].reshape((-1, 28*28)).astype(np.float32)
    Y_test = parse(fetch("http://yann.lecun.com/exdb/mnist/t10k-labels-idx1-ubyte.gz"))[8:]
    return map(tensor, (x_train/255,y_train,x_valid/255,y_valid))

In [27]:
#export
def normalize(x, m, s): return (x-m)/s

In [28]:
x_train,y_train,x_valid,y_valid = get_data()

In [29]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1307), tensor(0.3081))